In [67]:
import sagemaker
import boto3

In [68]:
sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = 'retocreditriskkgv'
print("Usando bucket:", bucket)

Usando bucket: retocreditriskkgv


In [75]:
sk_prefix = "sagemaker/reto_credit_risk/sklearncontainer"
trainpath = sess.upload_data(
    path="data/toTrain/train-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)
testpath = sess.upload_data(
    path="data/toTrain/test-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)

In [76]:
%%writefile script.py

import argparse

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score
import sklearn
import joblib
import os
import numpy as np
import pandas as pd

def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

if __name__== "__main__":
    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()
    
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state", type=int, default=0)
    
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train-V-1.csv")
    parser.add_argument("--test-file", type=str, default="test-V-1.csv")
    
    args, _ = parser.parse_known_args()
    
    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)
    
    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))
    
    features = list(train_df.columns)
    label = features.pop(-1)
    
    print("Building training and testing datasets")
    print()
    
    X_train = train_df[features]
    X_test = test_df[features]
    
    y_train = train_df[label]
    y_test = test_df[label]
    
    print('Column order: ')
    print(features)
    print()
    
    print('Label column is:', label)
    print()
    
    print("Data Shape: ")
    print()
    print("----SHAPE OF TRAINING DATA (80%)")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("----SHAPE OF TESTING DATA (20%)")
    print(X_test.shape)
    print(y_test.shape)
    print()
    
    print("Training RandomForest Model....")
    print()
    model = RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state, verbose=True)
    model.fit(X_train, y_train)
    print()
    
    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, model_path)
    print("Model persisted at "+ model_path)
    print()
    
    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred_test)
    test_rep = classification_report(y_test, y_pred_test)
    
    print()
    print("--- METRICS RESULTS FOR TESTING DATA ---")
    print()
    print("Total Rows are:", X_test.shape[0])
    print("[TESTING] Model accuracy is: ", test_acc)
    print("[TESTING] Testing Report: ")
    print(test_rep)
    

Overwriting script.py


In [77]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role="arn:aws:iam::940583586544:role/sagemaker_credit_risk",
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name = "RF-custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0,
    },
    use_spot_instances= True,
    max_wait = 7200,
    max_run = 3600
)

In [78]:
sklearn_estimator.fit({"train":trainpath, "test": testpath}, wait=True, logs=True)

[12/09/24 13:37:42] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=489577;file://C:\Users\kevin\anaconda3\envs\llm-aws\Lib\site-packages\sagemaker\telemetry\telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=443522;file://C:\Users\kevin\anaconda3\envs\llm-aws\Lib\site-packages\sagemaker\telemetry\telemetry_logging.py#90\90]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

[12/09/24 13:37:46] INFO     Creating training-job with name:                                       ]8;id=224662;file://C:\Users\kevin\anaconda3\envs\llm-aws\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=524276;file://C:\Users\kevin\anaconda3\envs\llm-aws\Lib\site-packages\sagemaker\session.py#1042\1042]8;;\
                             RF-custom-sklearn-2024-12-09-18-37-42-195                                             

2024-12-09 18:37:50 Starting - Starting the training job...
2024-12-09 18:38:04 Starting - Preparing the instances for training...
2024-12-09 18:38:40 Downloading - Downloading input data...
2024-12-09 18:39:06 Downloading - Downloading the training image...
2024-12-09 18:39:57 Training - Training image download completed. Training in progress.
2024-12-09 18:39:57 Uploading - Uploading generated training model2024-12-09 18:39:50,546 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-12-09 18:39:50,549 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-12-09 18:39:50,593 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-12-09 18:39:50,741 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-12-09 18:39:50,753 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-12-09 18:39:50,765 sagemaker-training-toolkit INFO

In [79]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model Artifact persisted at ", artifact)


2024-12-09 18:40:10 Starting - Preparing the instances for training
2024-12-09 18:40:10 Downloading - Downloading the training image
2024-12-09 18:40:10 Training - Training image download completed. Training in progress.
2024-12-09 18:40:10 Uploading - Uploading generated training model
2024-12-09 18:40:10 Completed - Training job completed
Model Artifact persisted at  s3://sagemaker-us-east-1-940583586544/RF-custom-sklearn-2024-12-09-18-37-42-195/output/model.tar.gz


In [80]:
artifact

's3://sagemaker-us-east-1-940583586544/RF-custom-sklearn-2024-12-09-18-37-42-195/output/model.tar.gz'

In [81]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "Credit-risk-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name = model_name,
    model_data = artifact,
    role="arn:aws:iam::940583586544:role/sagemaker_credit_risk",
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION
)

In [82]:
model

In [83]:
endpoint_name = "Credit-risk-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name
)

EndpointName=Credit-risk-model-2024-12-09-18-40-56


[12/09/24 13:41:00] INFO     Creating model with name: Credit-risk-model-2024-12-09-18-40-56        ]8;id=903689;file://C:\Users\kevin\anaconda3\envs\llm-aws\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=819848;file://C:\Users\kevin\anaconda3\envs\llm-aws\Lib\site-packages\sagemaker\session.py#4094\4094]8;;\

[12/09/24 13:41:02] INFO     Creating endpoint-config with name                                     ]8;id=922657;file://C:\Users\kevin\anaconda3\envs\llm-aws\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=480929;file://C:\Users\kevin\anaconda3\envs\llm-aws\Lib\site-packages\sagemaker\session.py#5889\5889]8;;\
                             Credit-risk-model-2024-12-09-18-40-56                                                 

                    INFO     Creating endpoint with name Credit-risk-model-2024-12-09-18-40-56      ]8;id=470697;file://C:\Users\kevin\anaconda3\envs\llm-aws\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=295534;file://C:\Users\kevin\anaconda3\envs\llm-aws\Lib\site-packages\sagemaker\session.py#4711\4711]8;;\

-------!